In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e' or base == 'Delta':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s])}

In [3]:
simp = True

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [4]:
# linear
def deltaBySpot_(s,v, b, w, j, e, Delta, a):
  f = (s - (v + b) * w) / w
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(*all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [5]:
# convex
def spotByExp_(s, v, b, w, j, e, Delta, a):
  f = a * ((1 + 1/j) ** e)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [6]:
# convex
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a):
    f = deltaBySpot[asset].subs(s, spotByExp[asset])
    if simp:
        f = simplify(f)
    return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b

In [7]:
# convex
eff = Delta_s / -Delta_b
eff

-Delta_s/Delta_b

In [9]:
Delta_b_by_Exp_s = deltaByExp['s'] * spotByExp['b'] / spotByExp['s']
if simp:
    Delta_b_by_Exp_s = simplify(Delta_b_by_Exp_s)
Delta_b_by_Exp_s

a_b*((j_b + 1)/j_b)**e_b*(j_s + 1)**(-e_s)*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))/(a_s*w_s)

In [10]:
Delta_s_by_Exp_b = deltaByExp['b'] * spotByExp['s'] / spotByExp['b']
if simp:
    Delta_s_by_Exp_b = simplify(Delta_s_by_Exp_b)
Delta_s_by_Exp_b

a_s*((j_s + 1)/j_s)**e_s*(j_b + 1)**(-e_b)*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b))/(a_b*w_b)

In [11]:
effByExp_b = eff.subs({Delta_b: deltaByExp['b'], Delta_s: Delta_s_by_Exp_b})
if simp:
    effByExp_b = simplify(effByExp_b)
effByExp_b

-a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [12]:
effByExp_s = eff.subs({Delta_b: Delta_b_by_Exp_s, Delta_s: deltaByExp['s']})
if simp:
    effByExp_s = simplify(effByExp_s)
effByExp_s

-a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [13]:
simplify(effByExp_b - effByExp_s)

0

In [14]:
effByExps = effByExp_b

In [ ]:
si

In [15]:
H = hessian(effByExps, [e_s, e_b])
H

Matrix([
[                -a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log((j_s + 1)/j_s)**2/a_b, -a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b],
[-a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b,                 -a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b]])

In [17]:
topLeft = simplify(H[0, 0])
topLeft

-a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log((j_s + 1)/j_s)**2/a_b

-> negative --> at best concave

In [19]:
det = H.det()
det

0

In [20]:
print(effByExps)

-a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b


### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- deltaBuyingByExp is negative => -deltaBuyingByExp is positive
- inversion of a positive function flips convexity/concavity => 1 / (-deltaBuyingByExp) is convex
- multiplying two nonnegative functions with different variables, that are each convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaSellingByExp / (-deltaBuyingByExp) is convex. qed


## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts (TODO)
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

In [ ]:
# convex
a0Buying = spotByExp['s'] * deltaByExp['b']
if simp:
    a0Buying = simplify(a0Buying)
a0Buying

In [ ]:
# convex
a0Selling = spotByExp['b'] * deltaByExp['s']
if simp:
    a0Selling = simplify(a0Selling)
a0Selling

In [ ]:

a0Bound = a0Buying - a0Selling # <=! 0
if simp:
    a0Bound = simplify(a0Bound)
a0Bound

In [ ]:
def expBound_(asset, s, v, b, w, j, e, Delta, a):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

In [ ]:
expBound['s']

In [ ]:
maxBuyingBound = -Delta_b - b_b # <=! 0
maxBuyingBound = maxBuyingBound.subs({Delta_b: deltaByExp['b']})
if simp:
    maxBuyingBound = simplify(maxBuyingBound)
maxBuyingBound

In [ ]:
max_s, max_I = symbols('max_s, max_I', positive=True, integer=True)

maxSellingBound = Delta_s - max_s # <=! 0
maxSellingBound = maxSellingBound.subs({Delta_s: deltaByExp['s']})
if simp:
    maxSellingBound = simplify(maxSellingBound)
maxSellingBound

In [ ]:
def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a):
  return spotByExp[asset] - max_I # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

## Lagrangian

In [ ]:
l1, l2, l3, l4, l5, l6, l7 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6 lambda_7', positive=True)

L = effByExps + + l1 * a0Bound + l2 * expBound['b'] + l3 * expBound['s'] + l4 * maxBuyingBound + l5 * maxSellingBound + l6 * maxSpotBound['b'] + l7 * maxSpotBound['s']
if simp:
    L = simplify(L)
L

In [ ]:
# Partial derivatives
dL_des = L.diff(e_s)
dL_deb = L.diff(e_b)
dL_dl1 = L.diff(l1)
dL_dl2 = L.diff(l2)
dL_dl3 = L.diff(l3)
dL_dl4 = L.diff(l4)
dL_dl5 = L.diff(l5)
dL_dl6 = L.diff(l6)
dL_dl7 = L.diff(l7)

# Set them to zero
stationary_conditions = [
    Eq(dL_des, 0),
    Eq(dL_deb, 0),
    Eq(dL_dl1, 0),
    Eq(dL_dl2, 0),
    Eq(dL_dl3, 0),
    Eq(dL_dl4, 0),
    Eq(dL_dl5, 0),
    Eq(dL_dl6, 0),
    Eq(dL_dl7, 0),
]

In [ ]:
solutions_ = solve(stationary_conditions, (e_s, e_b, l1, l2, l3, l4, l5, l6, l7))

print(len(solutions_))
solutions = solutions_[0]
e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol, l7_sol = solutions
# e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol = solutions
solutions

In [ ]:
e_s_sol

In [ ]:
e_b_sol

In [ ]:
l1_sol

In [ ]:
l2_sol

In [ ]:
l3_sol

In [ ]:
l4_sol

In [ ]:
l5_sol

In [ ]:
l6_sol

In [ ]:
l7_sol

In [ ]:
for sol in solutions:
  print(sol.free_symbols)